In [1]:
import numpy as np
import pandas as pd

**I implemented the DCC-GARCH model on the daily returns of S&P500 index and of JP Morgan Chase & Co.**

**The trading data from 2010/01/01 to 2019/12/31 was downloaded from Yahoo Finance.**

In [3]:
import efinance as ef
stock_code = '399905'
df = ef.stock.get_quote_history(stock_code)
stock_code = '000300'
df2 = ef.stock.get_quote_history(stock_code)
# 将日期列转换为日期时间类型
df['日期'] = pd.to_datetime(df['日期'])
# 设置日期列为索引
df.set_index('日期', inplace=True)
# 选择指定日期范围的数据
start_date = '2017-01-01'
end_date = '2022-12-31'
filtered_data = df.loc[start_date:end_date]
df = filtered_data
# 将日期列转换为日期时间类型
df2['日期'] = pd.to_datetime(df2['日期'])
# 设置日期列为索引
df2.set_index('日期', inplace=True)
# 选择指定日期范围的数据
start_date = '2017-01-01'
end_date = '2022-12-31'
filtered_data2 = df2.loc[start_date:end_date]
df2 = filtered_data2

In [6]:
sp = df
jpm = df2

In [7]:
sp_return = np.log(sp['收盘']).diff().dropna() # log return of S&P500 index
jpm_return = np.log(jpm['收盘']).diff().dropna() # log return of JP Morgan Chase & Co.

In [8]:
sp_return = sp_return.iloc[::-1] # the latest data should come first
jpm_return = jpm_return.iloc[::-1] # the latest data should come first

**In accord with _Multivariate DCC-GARCH Model (Elisabeth Orskaug, 2009)_, a GARCH model was first fitted for each log return separately and then a DCC model was fitted for the cross-correlation between the two time series. Additionally, I used threhold GARCH (usually abbreviated to TGARCH) in order to better capture the asymmetry of the stock volatility.**

**The structure of the entire model can be written down as follows:**

\begin{align}
&\left[\begin{array}{c}
     r_{sp,t} \\
     r_{jpm,t} 
     \end{array}\right]  \sim N(0,H_t) \\
&H_t  = 
     \left[\begin{array}{cc}
     \sigma_{sp,t} & 0 \\
     0 & \sigma_{jpm,t} 
     \end{array}\right]
     \left[\begin{array}{cc}
     1 & \rho_t \\
     \rho_t & 1 
     \end{array}\right]
     \left[\begin{array}{cc}
     \sigma_{sp,t} & 0 \\
     0 & \sigma_{jpm,t} 
     \end{array}\right] = 
     \left[\begin{array}{cc}
     \sigma_{sp,t} & 0 \\
     0 & \sigma_{jpm,t} 
     \end{array}\right] 
     R_t
     \left[\begin{array}{cc}
     \sigma_{sp,t} & 0 \\
     0 & \sigma_{jpm,t} 
     \end{array}\right]\\
&R_t  = 
     \left[\begin{array}{cc}
     \frac{1}{\sqrt{q_{11,t}}} & 0 \\
     0 & \frac{1}{\sqrt{q_{22,t}}}
     \end{array}\right] 
     \left[\begin{array}{cc}
     q_{11,t} & q_{12,t} \\
     q_{12,t} & q_{22,t}
     \end{array}\right] 
     \left[\begin{array}{cc}
     \frac{1}{\sqrt{q_{11,t}}} & 0 \\
     0 & \frac{1}{\sqrt{q_{22,t}}}
     \end{array}\right] =
     \left[\begin{array}{cc}
     \frac{1}{\sqrt{q_{11,t}}} & 0 \\
     0 & \frac{1}{\sqrt{q_{22,t}}}
     \end{array}\right]
     Q_t
     \left[\begin{array}{cc}
     \frac{1}{\sqrt{q_{11,t}}} & 0 \\
     0 & \frac{1}{\sqrt{q_{22,t}}}
     \end{array}\right]\\
&\sigma_{sp,t}^2 = \omega_{sp} + \alpha_{sp}\cdot r_{sp,t-1}^2 + \gamma_{sp}\cdot r_{sp,t-1}^2 \cdot I_{sp,t-1}^{-} + \beta_{sp} \cdot \sigma_{sp,t-1}^2\\
&\sigma_{jpm,t}^2 = \omega_{jpm} + \alpha_{jpm}\cdot r_{jpm,t-1}^2 + \gamma_{jpm}\cdot r_{jpm,t-1}^2 \cdot I_{jpm,t-1}^{-} + \beta_{jpm} \cdot \sigma_{jpm,t-1}^2\\
&Q_t = (1-a-b)\cdot\bar{Q} + a\cdot\epsilon_{t-1}\epsilon_{t-1}' + b\cdot Q_{t-1}\\
&\bar{Q} = \dfrac{1}{T}\sum_{t=1}^{T}\epsilon_{t}\epsilon_{t}', \quad \epsilon_{t} = \left[\begin{array}{c}
     r_{sp,t}/\sigma_{sp,t} \\
     r_{jpm,t}/\sigma_{jpm,t}
     \end{array}\right]
\end{align}

**The parameters to be optimized are**

$\omega_{sp},\alpha_{sp},\gamma_{sp},\beta_{sp}$ **for the TGARCH(1,1,1) model fitting on the log return of S&P500 index,**

$\omega_{jpm},\alpha_{jpm},\gamma_{jpm},\beta_{jpm}$ **for the TGARCH(1,1,1) model fitting on the log return of JP Morgan Chase & Co.,**

**and $a,b$ for the DCC model fitting on the cross-correlation.**

In [9]:
from DCC_GARCH.GARCH.GARCH import GARCH
from DCC_GARCH.GARCH.GARCH_loss import garch_loss_gen

In [10]:
sp_model = GARCH(1,1)
sp_model.set_loss(garch_loss_gen(1, 1))
sp_model.set_max_itr(1)

In [11]:
sp_model.fit(sp_return)


   NFVALS =    5   F =-4.955337E+03    MAXCV =-0.000000E+00
   X = 5.000000E-03   1.000000E-01   1.000000E-01   8.500000E-01
Iteration: 1. Training loss: -4.955E+03.


[-4955.336772171043]

In [12]:
sp_model.get_theta()

array([0.005, 0.1  , 0.1  , 0.85 ])

**Therefore, we have the fitted TGARCH(1,1,1) on the log return of S&P500 index:**
\begin{align}
\omega_{sp}&= 0.005\\
\alpha_{sp}&= 0.1\\
\gamma_{sp}&= 0.1\\
\beta_{sp}&= 0.85
\end{align}

In [13]:
jpm_model = GARCH(1,1)
jpm_model.set_loss(garch_loss_gen(1, 1))
jpm_model.set_max_itr(1)

In [14]:
jpm_model.fit(jpm_return)


   NFVALS =    5   F =-4.959110E+03    MAXCV =-0.000000E+00
   X = 5.000000E-03   1.000000E-01   1.000000E-01   8.500000E-01
Iteration: 1. Training loss: -4.959E+03.


[-4959.109938173486]

In [15]:
jpm_model.get_theta()

array([0.005, 0.1  , 0.1  , 0.85 ])

**Therefore, we have the fitted TGARCH(1,1,1) on the log return of JP Morgan Chase & Co.:**
\begin{align}
\omega_{jpm}&= 7.416e-30\\
\alpha_{jpm}&= 0.0134\\
\gamma_{jpm}&= 0.0311\\
\beta_{jpm} &= 0.9711
\end{align}

In [16]:
sp_sigma = sp_model.sigma(sp_return)
sp_epsilon = sp_return / sp_sigma

In [17]:
jpm_sigma = jpm_model.sigma(jpm_return)
jpm_epsilon = jpm_return / jpm_sigma

In [18]:
epsilon = np.array([sp_epsilon,jpm_epsilon])

In [31]:
#使用 sp_model.sigma(sp_return) 计算了波动率，其中 sp_return 是你的收益率序列。
#将收益率序列 sp_return 除以波动率，得到了标准化残差 sp_epsilon。
#这些标准化残差通常被用来进行进一步的模型诊断或分析。如果模型是正确的，标准化残差应该是平稳的，没有明显的结构。
# Ljung-Box检验
import statsmodels.api as sm
lb_test = sm.stats.acorr_ljungbox(sp_epsilon, lags=20, return_df=True)
print("Ljung-Box统计量和p-value：")
print(lb_test)
#Ljung-Box检验的零假设是残差序列是白噪声，即在所有滞后阶上都没有自相关。因此，你关注的主要是p-value是否小于选择的显著性水平（通常选择0.05）。
#在你的结果中，对于每个滞后阶，lb_pvalue 都相对较大，远远大于0.05。这意味着在这个检验中，你不能拒绝残差序列是白噪声的零假设。这是好的，因为白噪声表示残差序列没有明显的自相关结构，表明你的模型捕捉了数据中的信息。

Ljung-Box统计量和p-value：
      lb_stat  lb_pvalue
1    0.074128   0.785418
2    0.100976   0.950765
3    1.151658   0.764621
4    4.870020   0.300896
5    5.355427   0.374059
6    7.268196   0.296755
7    7.434641   0.385069
8    8.264790   0.408041
9    8.305893   0.503638
10   9.036949   0.528601
11   9.807177   0.547811
12  10.834895   0.543119
13  10.865917   0.622049
14  13.009323   0.525790
15  13.308190   0.578507
16  13.342712   0.647554
17  13.343398   0.712911
18  14.076942   0.724062
19  14.625705   0.746072
20  14.937361   0.779981


In [32]:
#使用 sp_model.sigma(sp_return) 计算了波动率，其中 sp_return 是你的收益率序列。
#将收益率序列 sp_return 除以波动率，得到了标准化残差 sp_epsilon。
#这些标准化残差通常被用来进行进一步的模型诊断或分析。如果模型是正确的，标准化残差应该是平稳的，没有明显的结构。
# Ljung-Box检验
lb_test = sm.stats.acorr_ljungbox(jpm_epsilon, lags=20, return_df=True)
print("Ljung-Box统计量和p-value：")
print(lb_test)
#Ljung-Box检验的零假设是残差序列是白噪声，即在所有滞后阶上都没有自相关。因此，你关注的主要是p-value是否小于选择的显著性水平（通常选择0.05）。
#在你的结果中，对于每个滞后阶，lb_pvalue 都相对较大，远远大于0.05。这意味着在这个检验中，你不能拒绝残差序列是白噪声的零假设。这是好的，因为白噪声表示残差序列没有明显的自相关结构，表明你的模型捕捉了数据中的信息。

Ljung-Box统计量和p-value：
      lb_stat  lb_pvalue
1    0.012688   0.910314
2    0.106363   0.948208
3    2.530996   0.469716
4    5.756671   0.218072
5    7.246554   0.202940
6   10.608356   0.101261
7   11.666141   0.112086
8   11.852392   0.157912
9   11.938554   0.216796
10  13.978502   0.173974
11  14.019919   0.231893
12  14.425345   0.274375
13  15.293578   0.289386
14  16.197307   0.301474
15  16.220807   0.367530
16  16.320024   0.430857
17  16.542127   0.485780
18  16.981906   0.524349
19  17.244767   0.573290
20  18.935481   0.526023


In [47]:
# 假设模拟的时间步数为 n_steps
n_steps = 10  # 可以根据需要调整
# 使用模型参数进行波动率预测
predicted_volatility1 = sp_model.sigma(sp_return)
# 模拟未来数据
simulated_returns1 = np.random.normal(0, np.sqrt(predicted_volatility1[-1]), size=n_steps)
# 打印模拟的未来数据
print("模拟的未来数据：", simulated_returns1)

模拟的未来数据： [ 0.1885295  -0.12900626  0.18861878  0.07905482  0.10797667  0.07280456
  0.05230595  0.01823324  0.24737227 -0.15161747]


In [44]:
# 假设模拟的时间步数为 n_steps
n_steps = 10  # 可以根据需要调整
# 使用模型参数进行波动率预测
predicted_volatility = jpm_model.sigma(jpm_return)
# 模拟未来数据
simulated_returns = np.random.normal(0, np.sqrt(predicted_volatility[-1]), size=n_steps)
# 打印模拟的未来数据
print("模拟的未来数据：", simulated_returns)

模拟的未来数据： [-0.07274476  0.01905556  0.05023988 -0.11001262 -0.05821835 -0.10711643
 -0.16911011  0.10988806 -0.0206373   0.08797078]


In [73]:
# 假设模拟的时间步数为 n_steps
n_steps = 10  # 可以根据需要调整
# 获取训练好的模型参数
trained_theta = sp_model.get_theta()
# 初始化数组用于保存模拟的收益率
simulated_returns = np.zeros(n_steps)
# 使用训练好的参数进行 GARCH 过程的模拟
for i in range(n_steps):
    # 使用模型的 sigma 方法计算当前时刻的波动率
    current_volatility = sp_model.sigma(simulated_returns[:i+1])[-1]
    # 使用正态分布生成模拟的收益率
    simulated_returns[i] = np.random.normal(0, np.sqrt(current_volatility))
# 获取实际的收益率序列（假设已有 jpm_return）
actual_returns = sp_return[-n_steps:]
# 计算均方误差 (MSE)
mse = mean_squared_error(actual_returns, simulated_returns)
print("拟合效果的均方误差 (MSE):", mse)
#平均绝对误差 MAR
def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
#均方根误差 (RMSE)
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))
mae = mean_absolute_error(actual_returns, simulated_returns)
print("拟合效果的均方误差 (MSE):", mae)
rmse = root_mean_squared_error(actual_returns, simulated_returns)
print("拟合效果的均方误差 (MSE):", rmse)

拟合效果的均方误差 (MSE): 0.0001337967992694452
拟合效果的均方误差 (MSE): 0.009367278262215884
拟合效果的均方误差 (MSE): 0.01156705663811867


In [74]:
# 假设模拟的时间步数为 n_steps
n_steps = 10  # 可以根据需要调整
# 获取训练好的模型参数
trained_theta = jpm_model.get_theta()
# 初始化数组用于保存模拟的收益率
simulated_returns = np.zeros(n_steps)
# 使用训练好的参数进行 GARCH 过程的模拟
for i in range(n_steps):
    # 使用模型的 sigma 方法计算当前时刻的波动率
    current_volatility = jpm_model.sigma(simulated_returns[:i+1])[-1]
    # 使用正态分布生成模拟的收益率
    simulated_returns[i] = np.random.normal(0, np.sqrt(current_volatility))
# 获取实际的收益率序列（假设已有 jpm_return）
actual_returns = jpm_return[-n_steps:]
# 计算均方误差 (MSE)
mse = mean_squared_error(actual_returns, simulated_returns)
print("拟合效果的均方误差 (MSE):", mse)
mae = mean_absolute_error(actual_returns, simulated_returns)
print("拟合效果的均方误差 (MSE):", mae)
rmse = root_mean_squared_error(actual_returns, simulated_returns)
print("拟合效果的均方误差 (MSE):", rmse)

拟合效果的均方误差 (MSE): 2.0357634310928e-05
拟合效果的均方误差 (MSE): 0.0035517319723680886
拟合效果的均方误差 (MSE): 0.004511943518144703


In [19]:
from DCC_GARCH.DCC.DCC import DCC
from DCC_GARCH.DCC.DCC_loss import dcc_loss_gen

In [20]:
dcc_model = DCC()
dcc_model.set_loss(dcc_loss_gen())

In [22]:
dcc_model.get_ab()

array([0.78338396, 0.21630562])

**Therefore, we have the fitted DCC(1,1) for the cross-correlation between the log returns of S&P500 index and of JP Morgan Chase & Co.:**
\begin{align}
a &= 0.0138\\
b&= 0.9636
\end{align}

In [62]:
epsilon

array([[ 0.00659986, -0.0043875 , -0.02193689, ..., -0.04469405,
         0.01002392,  0.86051022],
       [ 0.02114337, -0.02061813, -0.02339581, ..., -0.06192752,
        -0.00215506,  0.63714546]])

In [66]:
train_losses = dcc_model.fit(epsilon)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2893.8177190860297
            Iterations: 5
            Function evaluations: 3
            Gradient evaluations: 1
Iteration: 1. Training loss: -2.894E+03.
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2893.8177190860297
            Iterations: 5
            Function evaluations: 3
            Gradient evaluations: 1
Iteration: 2. Training loss: -2.894E+03.


In [67]:
print("训练损失:", train_losses)

训练损失: [-2893.8177190860297, -2893.8177190860297]


In [72]:
# 使用 DCC 模型的损失函数计算生成的时间序列的损失
loss_for_epsilon = dcc_model.get_loss_func()(epsilon)
print("给定 epsilon 的损失:", loss_for_epsilon)

给定 epsilon 的损失: <function dcc_loss_gen.<locals>.loss1.<locals>.loss at 0x144a81f80>


In [71]:
future_data_dcc

<function DCC_GARCH.DCC.DCC_loss.dcc_loss_gen.<locals>.loss1.<locals>.loss(ab)>